In [4]:
from pydantic import BaseModel, ValidationError, condecimal
from typing import List
from datetime import datetime
import pandas as pd

# Определяем модель Pydantic
class Transaction(BaseModel):
    sender_account: str
    receiver_account: str
    amount: condecimal(gt=0)
    currency: str
    timestamp: datetime


# Загружаем CSV
df = pd.read_csv("data/transactions.csv")

valid_rows = []
invalid_rows = []

# Проверяем каждую строку через Pydantic
for i, row in df.iterrows():
    try:
        tx = Transaction(**row.to_dict())
        valid_rows.append(row)
    except ValidationError as e:
        error_row = row.copy()
        error_row["error"] = str(e)
        invalid_rows.append(error_row)

# Преобразуем в отдельные DataFrame'ы
df_valid = pd.DataFrame(valid_rows)
df_invalid = pd.DataFrame(invalid_rows)

# Сохраняем результат
df_valid.to_csv("output/valid_transactions.csv", index=False)
df_invalid.to_csv("output/invalid_transactions.csv", index=False)

print(f"✅ Всего строк: {len(df)}")
print(f"✅ Валидные: {len(df_valid)}")
print(f"❌ Невалидные: {len(df_invalid)} (сохранены в output/invalid_transactions.csv)")


✅ Всего строк: 100000
✅ Валидные: 0
❌ Невалидные: 100000 (сохранены в output/invalid_transactions.csv)


In [5]:
import os

os.remove("output/valid_transactions.csv")
os.remove("output/invalid_transactions.csv")
